In [0]:
--=========================================================
-- Silver Account Table
--=========================================================
CREATE OR REFRESH LIVE TABLE silver_account (
  CONSTRAINT account_id_not_null EXPECT (account_id IS NOT NULL) ON VIOLATION DROP ROW,
  CONSTRAINT account_name_not_null EXPECT (account_name IS NOT NULL),
  CONSTRAINT valid_created_date EXPECT (created_datetime IS NOT NULL)
)
COMMENT "Cleaned and standardized account dimension from Salesforce. Source columns with spaces are quoted."
TBLPROPERTIES ("quality" = "silver")
AS
SELECT
  -- Identifiers (Quoting source columns with spaces)
  CAST(`Account ID` AS STRING) AS account_id,
  CAST(`Owner ID` AS STRING) AS owner_user_id, 
  CAST(`Parent Account ID` AS STRING) AS parent_account_id,

  -- Core Attributes
  TRIM(CAST(`Account Name` AS STRING)) AS account_name,
  CAST(`Account Number` AS STRING) AS account_number,
  CAST(`Account Type` AS STRING) AS account_type,
  CAST(`Industry` AS STRING) AS industry,
  CAST(`Account Rating` AS STRING) AS account_rating,
  CAST(`Account Site` AS STRING) AS account_site,
  CAST(`Ownership` AS STRING) AS account_ownership,

  -- Billing/Location
  TRIM(CAST(`Billing City` AS STRING)) AS billing_city,
  CAST(`Billing State/Province` AS STRING) AS billing_state_province,
  CAST(`Billing Country` AS STRING) AS billing_country,

  -- Shipping/Location
  TRIM(CAST(`Shipping City` AS STRING)) AS shipping_city,
  CAST(`Shipping State/Province` AS STRING) AS shipping_state_province,
  CAST(`Shipping Country` AS STRING) AS shipping_country,
  CAST(`Shipping Zip/Postal Code` AS STRING) AS shipping_zip,


  -- Dates (Ensure source format is compatible or use to_timestamp(`Col Name`, format))
  CAST(`Created Date` AS TIMESTAMP) AS created_datetime,
  CAST(`Last Modified Date` AS TIMESTAMP) AS last_modified_datetime,
  CAST(`Last Modified By ID` AS STRING) AS last_modified_ByID,
  CAST(`Last Activity` AS DATE) AS last_activity_date, -- Assuming Last Activity is Date

  -- Flags/Status
  CAST(`Is Demo Center` AS BOOLEAN) AS is_demo_center,
  CAST(`Life Science KeyAccount` AS BOOLEAN) AS is_life_science_key_account,
  CAST(`Deleted` AS BOOLEAN) AS is_deleted

FROM aitest.crm_bronze.dlt_account; -- Ensure Bronze table name is correct

In [0]:
--=========================================================
-- Silver Lead Table
--=========================================================
CREATE OR REFRESH LIVE TABLE silver_lead (
  CONSTRAINT lead_id_not_null EXPECT (lead_id IS NOT NULL) ON VIOLATION DROP ROW,
  -- CONSTRAINT valid_email EXPECT (email IS NULL OR email LIKE '%@%.%'),
  CONSTRAINT valid_status EXPECT (lead_status IS NOT NULL)
)
COMMENT "Cleaned and standardized lead dimension from Salesforce. Source columns with spaces are quoted."
TBLPROPERTIES ("quality" = "silver")
AS
SELECT
  -- Identifiers
  CAST(`Lead ID` AS STRING) AS lead_id,
  CAST(`Converted Account ID` AS STRING) AS converted_account_id,
  CAST(`Converted Opportunity ID` AS STRING) AS converted_opportunity_id,
  CAST(`Companion Lead` AS STRING) AS companion_lead, 

  -- Core Attributes
  TRIM(CAST(`Organization` AS STRING)) AS company_name, 
  CAST(`Lead Source` AS STRING) AS lead_source,
  CAST(`Status` AS STRING) AS lead_status,
  CAST(`Industry` AS STRING) AS industry,
  CAST(`Lead Type` AS STRING) AS lead_type,
  CAST(`LeadRecordType` AS STRING) AS lead_record_type,
  CAST(`Lead Status at Conversion` AS STRING) AS status_conversion, 


  -- Contact Info
  -- LOWER(TRIM(CAST(`Secondary Email` AS STRING))) AS email, 
  -- CAST(`Phone` AS STRING) AS phone,
  -- CAST(`Website` AS STRING) AS website,

  -- Location
  TRIM(CAST(`City` AS STRING)) AS city,
  CAST(`State/Province` AS STRING) AS state_province,
  CAST(`Country` AS STRING) AS country,
  -- CAST(`Postal Code` AS STRING) AS postal_code, 

  -- Dates
  CAST(`Created Date` AS TIMESTAMP) AS created_datetime,
  -- CAST(`Last Modified Date` AS TIMESTAMP) AS last_modified_datetime, 
  CAST(`Pardot Conversion Date` AS TIMESTAMP) AS converted_datetime, 
  CAST(`Pardot Last Activity` AS DATE) AS last_activity_date, 

  -- Flags/Status
  CAST(`Converted` AS BOOLEAN) AS is_converted,
  CAST(`Opted Out of Email` AS BOOLEAN) AS has_opted_out_email, 
  CAST(`Other Dead Reason` AS STRING) AS dead_reason, 
  CAST(`Key Account` AS BOOLEAN) AS is_key_account,

  -- Scores
  CAST(`Lead Score` AS INT) AS lead_score,
  CAST(`Pardot Score` AS INT) AS pardot_score,

  -- -- Marketing Context
  CAST(`Marketing Segmentation` AS STRING) AS mark_segmentation,
  CAST(`Mass Spec Manufacturer` AS STRING) AS mass_spec_manufacturer,
  CAST(`Mass Spec Type` AS STRING) AS mass_spec_type
  -- CAST(`Google Analytics Content` AS STRING) AS ga_content,
  -- CAST(`Google Analytics Term` AS STRING) AS ga_term,
  -- CAST(`LeadConSource` AS STRING) AS lead_conversion_source,
  -- CAST(`Deleted` AS BOOLEAN) AS is_deleted -- Assuming Deleted exists

FROM aitest.crm_bronze.dlt_lead; 


In [0]:
--=========================================================
-- Silver Opportunity Table
--=========================================================
CREATE OR REFRESH LIVE TABLE silver_opportunity (
  CONSTRAINT opportunity_id_not_null EXPECT (opportunity_id IS NOT NULL) ON VIOLATION DROP ROW,
  CONSTRAINT account_id_not_null EXPECT (account_id IS NOT NULL),
  CONSTRAINT valid_close_date EXPECT (close_date IS NOT NULL),
  CONSTRAINT valid_stage EXPECT (stage_name IS NOT NULL)
)
COMMENT "Cleaned and standardized opportunity dimension from Salesforce. Source columns with spaces are quoted."
TBLPROPERTIES ("quality" = "silver")
AS
SELECT
  -- Identifiers
  CAST(`Opportunity ID` AS STRING) AS opportunity_id,
  CAST(`Account ID` AS STRING) AS account_id,
  CAST(`Owner ID` AS STRING) AS owner_user_id,
  CAST(`Primary Contact` AS STRING) AS primary_contact_id, -- Assuming Primary Contact stores the ID

  -- Core Attributes
  CAST(`Created By ID` AS STRING) AS created_by_id, 
  CAST(`Opportunity Type` AS STRING) AS opportunity_type,
  CAST(`Stage` AS STRING) AS stage_name,
  CAST(`Forecast Category` AS STRING) AS forecast_category,
  CAST(`Lead Source` AS STRING) AS lead_source,

  -- Financials
  CAST(`Amount` AS DOUBLE) AS amount,
  CAST(`Expected Amount` AS DOUBLE) AS expected_amount,
  CAST(`Probability (%)` AS DOUBLE) AS probability_percent,
  CAST(`Backlog Rev` AS DOUBLE) AS backlog_revenue,

  -- Dates
  CAST(`Created Date` AS TIMESTAMP) AS created_datetime,
  CAST(`Close Date` AS DATE) AS close_date,
  CAST(`Last Modified Date` AS TIMESTAMP) AS last_modified_datetime,
  CAST(`Last Stage Change Date` AS TIMESTAMP) AS last_stage_change_datetime,

  -- Status/Outcome
  CAST(`Closed` AS BOOLEAN) AS is_closed,
  CAST(`Won` AS BOOLEAN) AS is_won,
  CAST(`Closed Lost Reason` AS STRING) AS closed_lost_reason,
  CAST(`Opportunity status` AS STRING) AS opportunity_status,
  CAST(`Order Finalized` AS BOOLEAN) AS is_order_finalized,

  -- Product/Context
  CAST(`Product Category` AS STRING) AS product_category,
  CAST(`Product of Interest` AS STRING) AS product_of_interest,
  CAST(`Industry` AS STRING) AS industry,

  -- Flags
  CAST(`Has Line Item` AS BOOLEAN) AS has_line_item,
  CAST(`Has Open Activity` AS BOOLEAN) AS has_open_activity,
  CAST(`Has Overdue Task` AS BOOLEAN) AS has_overdue_task,
  CAST(`Deleted` AS BOOLEAN) AS is_deleted

FROM aitest.crm_bronze.dlt_opportunityTable;

In [0]:
--=========================================================
-- Silver Opportunity Product (Line Item) Table
--=========================================================
CREATE OR REFRESH LIVE TABLE silver_opportunity_product (
  CONSTRAINT line_item_id_not_null EXPECT (line_item_id IS NOT NULL) ON VIOLATION DROP ROW,
  CONSTRAINT opportunity_id_not_null EXPECT (opportunity_id IS NOT NULL),
  CONSTRAINT product_id_not_null EXPECT (product_id IS NOT NULL),
  CONSTRAINT valid_quantity EXPECT (quantity > 0),
  CONSTRAINT valid_total_price EXPECT (total_price IS NOT NULL)
)
COMMENT "Cleaned and standardized opportunity line items. Source columns with spaces are quoted."
TBLPROPERTIES ("quality" = "silver")
AS
SELECT
  -- Identifiers
  CAST(`Line Item ID` AS STRING) AS line_item_id,
  CAST(`Opportunity ID` AS STRING) AS opportunity_id,
  CAST(`Product ID` AS STRING) AS product_id,
  CAST(`Price Book Entry ID` AS STRING) AS price_book_entry_id,

  -- Financials
  CAST(`Quantity` AS DOUBLE) AS quantity,
  CAST(`List Price` AS DOUBLE) AS list_price,
  CAST(`Sales Price` AS DOUBLE) AS sales_price,
  CAST(`Discount` AS DOUBLE) AS discount_percent, 
  CAST(`Total Price` AS DOUBLE) AS total_price,
  CAST(`Subtotal` AS DOUBLE) AS subtotal, 

  -- Product Info
  CAST(`Product Code` AS STRING) AS product_code,
  TRIM(CAST(`Product Name` AS STRING)) AS product_name_at_sale,

  -- Dates
  CAST(`Created Date` AS TIMESTAMP) AS created_datetime,
  CAST(`Last Modified Date` AS TIMESTAMP) AS last_modified_datetime,
  CAST(`Date` AS DATE) AS service_date, 

  -- Status
  CAST(`Deleted` AS BOOLEAN) AS is_deleted,

  -- Other
  CAST(`Sort Order` AS INT) AS sort_order 


FROM aitest.crm_bronze.dlt_opportunityproduct;

In [0]:
--=========================================================
-- Silver User Table
--=========================================================
CREATE OR REFRESH LIVE TABLE silver_user (
  -- CONSTRAINT user_id_not_null EXPECT (user_id IS NOT NULL) ON VIOLATION DROP ROW
  -- CONSTRAINT valid_email EXPECT (email IS NULL OR email LIKE '%@%.%')
)
COMMENT "Cleaned and standardized user dimension from Salesforce. Source columns with spaces are quoted."
TBLPROPERTIES ("quality" = "silver")
AS
SELECT
  -- Identifiers
  CAST(`Individual ID` AS STRING) AS user_id, 
  CAST(`Manager ID` AS STRING) AS manager_user_id,

  -- Core Attributes
  CAST(`Full Name` AS STRING) AS full_name,
  -- CAST(`First Name` AS STRING) AS first_name, 
  -- CAST(`Last Name` AS STRING) AS last_name,
  -- CAST(`Title` AS STRING) AS title, 
  CAST(`Department` AS STRING) AS department,
  -- LOWER(TRIM(CAST(`Email` AS STRING))) AS email, 

  -- Status/Role
  CAST(`Active` AS BOOLEAN) AS is_active,
  CAST(`Profile ID` AS STRING) AS profile_id,
  -- CAST(`User Role ID` AS STRING) AS user_role_id, 

  -- Location/Locale
  TRIM(CAST(`City` AS STRING)) AS city,
  TRIM(CAST(`Street` AS STRING)) AS street,
  CAST(`Country` AS STRING) AS country,
  CAST(`State/Province` AS STRING) AS state_province, 
  CAST(`Locale` AS STRING) AS locale,
  CAST(`Language` AS STRING) AS language,

  -- Flags
  CAST(`Marketing User` AS BOOLEAN) AS is_marketing_user,
  CAST(`Offline User` AS BOOLEAN) AS is_offline_user,
  CAST(`Is Approver` AS BOOLEAN) AS is_approver,

  -- Dates
  CAST(`Created Date` AS TIMESTAMP) AS created_datetime,
  CAST(`System Modstamp` AS TIMESTAMP) AS system_modstamp,
  CAST(`Last Modified Date` AS TIMESTAMP) AS last_modified_datetime,
  CAST(`Last Login` AS TIMESTAMP) AS last_login_datetime

FROM aitest.crm_bronze.dlt_usertable; -- Ensure Bronze table name is correct